In [12]:
%load_ext autoreload
%autoreload 2

### Pennylane Example

QNode(wrapper): 양자 회로를 정의 & 실행
* 양자 디바이스 선택
* 회로 정의
* 실행 & 자동미분(pytorch/jax/tf 연동 가능)

Pennylane은 projective 뿐만 아니라 POVM도 지원함

In [13]:
import pennylane as qml
from pennylane import numpy as np

# 디바이스 선택 (여기서는 기본 시뮬레이터 사용)
dev = qml.device("default.qubit", wires=1)

# QNode 정의
@qml.qnode(dev)
def circuit(theta):
    qml.RX(theta, wires=0)           # 회전 게이트
    return qml.expval(qml.PauliZ(0)) # Z-기댓값 반환

print(circuit(0.1))  # 실행

0.9950041652780258


예시: **2원(binary) POVM**

$$E_0 = \frac{1}{2} |0\rangle\langle 0| + \frac{1}{2} |1\rangle\langle 1|, \quad  
E_1 = I - E_0$$

In [14]:
import pennylane as qml
from pennylane import numpy as np

n = 2
dev = qml.device("lightning.qubit", wires=n, shots=None)

prjs = [0,1]

def prepare(theta):
    # 예시: 0번 큐빗에만 상태 준비
    qml.RY(theta, wires=0)

@qml.qnode(dev)
def povm_probs(theta):
    prepare(theta)
    qml.Hadamard(wires=1)
    return [qml.expval(qml.Projector([P], wires=[1])) for P in prjs]  # [p0, p1, p2]

for theta in np.linspace(0, 1, 11):
    p0, p1 = povm_probs(theta)
    print(p0, p1)

0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.49999999999999994 0.49999999999999994
0.49999999999999983 0.49999999999999983
0.49999999999999994 0.49999999999999994
0.4999999999999999 0.4999999999999999
0.49999999999999994 0.49999999999999994
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999


$$E_0 = p_0 |0\rangle\langle 0| + p_1 |1\rangle\langle 1|, \quad  
E_1 = I - E_0$$

In [15]:
# 디바이스: 2큐빗(시스템 0, 보조 1). shots=None이면 확률을 정확히 계산(qml.probs)
dev = qml.device("default.qubit", wires=2, shots=None)

SYSTEM, ANC = 0, 1

def prepare_system(theta=0.0, phi=0.0):
    """단일 큐빗 순수상태 |psi> = cos(theta/2)|0> + e^{i phi} sin(theta/2)|1>"""
    qml.RY(theta, wires=SYSTEM)
    qml.RZ(phi, wires=SYSTEM)


@qml.qnode(dev)
def povm_diag_probs(p0, p1, theta=0.0, phi=0.0):
    """
    E0 = p0|0><0| + p1|1><1|, E1 = I - E0 의 Naimark dilation.
    반환: 보조(ANC) 측정 확률 [Pr(anc=0), Pr(anc=1)] = [tr(E0 rho), tr(E1 rho)]
    """
    # 유효성 검사
    if not (0.0 <= p0 <= 1.0 and 0.0 <= p1 <= 1.0):
        raise ValueError("p0, p1은 [0,1] 범위여야 합니다.")

    # 시스템 상태 준비 (원하는 |psi>로 바꾸세요)
    prepare_system(theta, phi)

    # 보조는 |0>에서 시작 (기본값)

    # 분기별 회전각: cos^2(theta_b) = p_b
    th0 = np.arccos(np.sqrt(p0))
    th1 = np.arccos(np.sqrt(p1))

    # uniformly-controlled R_y(2*theta_b)를 CRY와 X-sandwich로 구현
    # (원래 'control=|0>' 분기는 X로 뒤집어서 CRY, 다시 언컴퓨트)
    qml.PauliX(SYSTEM)
    qml.CRY(2 * th0, wires=[SYSTEM, ANC])  # 원래 |0> 분기
    qml.PauliX(SYSTEM)
    qml.CRY(2 * th1, wires=[SYSTEM, ANC])  # |1> 분기

    # 보조 Z-측정 확률 = 원하는 POVM의 결과 확률
    return qml.probs(wires=ANC)

# 사용 예시 ----------------------------------------------------------
# 시스템을 임의의 순수상태로 준비(여기선 theta=1.1, phi=0.3)
theta_sys, phi_sys = np.pi, 0.0

# 예: p0=0.9, p1=0.2  ->  E0=diag(0.9, 0.2)
probs = povm_diag_probs(p0=0.9, p1=0.2, theta=theta_sys, phi=phi_sys)
print("Pr[E0], Pr[E1] =", probs)  # [tr(E0 rho), tr(E1 rho)]

Pr[E0], Pr[E1] = [0.2 0.8]


$$P_{\rho_0} = \eta_0 \;\; P_{\rho_1} = \eta_1$$

In [16]:
# Ancilla 없는 구현
dev = qml.device("default.qubit", wires=1, shots=None)

def helstrom_unitary(psi0, psi1, eta0=0.5, eta1=0.5):
    # rho0, rho1
    rho0 = np.outer(psi0, psi0.conj())
    rho1 = np.outer(psi1, psi1.conj())
    # Gamma = eta0*rho0 - eta1*rho1
    Gamma = eta0 * rho0 - eta1 * rho1
    # 고유분해: Gamma v_j = lambda_j v_j
    vals, vecs = np.linalg.eigh(Gamma)  # vecs[:,j] = v_j (정규직교)
    # 양의 고유값에 대응하는 벡터를 첫번째 열로 배치
    idx = np.argsort(vals)[::-1]        # 내림차순: +가 먼저
    V = vecs[:, idx]                    # 열: [v_plus, v_minus]
    # 우리가 회로에서 쓸 유니터리: W^\dagger (Z측정 기준으로 보낼 것)
    Wdag = V.conj().T                   # W = V,  W|0>=v_plus
    return Wdag

@qml.qnode(dev)
def helstrom_probs(psi, psi0, psi1, eta0=0.5, eta1=0.5):
    # 입력 상태 준비 (임의의 |psi>)
    # psi는 길이 2의 복소벡터, 정규화 가정
    qml.StatePrep(psi, wires=0)
    # 헬스트롬 기저로 회전: W^\dagger
    Wdag = helstrom_unitary(psi0, psi1, eta0, eta1)
    qml.QubitUnitary(Wdag, wires=0)
    # Z측정 확률: [Pr(E0), Pr(E1)]
    return qml.probs(wires=0)

# 예시 사용:
psi0 = np.array([1,0], dtype=complex)                 # |0>
psi1 = (np.array([1,1],complex)/np.sqrt(2))           # |+>
psi  = (np.array([np.sqrt(0.0), np.sqrt(1.0)],complex))  # 임의 입력
print(helstrom_probs(psi, psi0, psi1, 0.5, 0.5))  # [Pr(E0), Pr(E1)]
# psi가 [1, 0]일 때, Pr(E0) 값이 Pr_succ 값과 일치하면 성공

[0.14644661 0.85355339]


$$U \;=\;  
\begin{pmatrix}  
\sqrt{E_0} & -\sqrt{E_1}\\[2pt]  
\sqrt{E_1} & \ \sqrt{E_0}  
\end{pmatrix}  
\quad(\text{블록 }2\times2)$$

$$U\bigl(|\psi\rangle\otimes|0\rangle_A\bigr)  
= \bigl(\sqrt{E_0}|\psi\rangle\bigr)\otimes|0\rangle_A  
+ \bigl(\sqrt{E_1}|\psi\rangle\bigr)\otimes|1\rangle_A.$$

In [17]:
# Ancilla 사용
dev2 = qml.device("default.qubit", wires=2, shots=None)
S, A = 0, 1

def helstrom_E0(psi0, psi1, eta0=0.5, eta1=0.5):
    # rho0, rho1
    rho0 = np.outer(psi0, psi0.conj())
    rho1 = np.outer(psi1, psi1.conj())
    # Gamma = eta0*rho0 - eta1*rho1
    Gamma = eta0 * rho0 - eta1 * rho1
    # 고유분해: Gamma v_j = lambda_j v_j
    vals, vecs = np.linalg.eigh(Gamma)  # vecs[:,j] = v_j (정규직교)
    print("Helstrom bound is ",0.5*(1 + np.sum(np.abs(vals))))
    # 양의 고유값에 대응하는 벡터를 첫번째 열로 배치
    V = vecs[:, vals >1e-12]                    # 열: [v_plus, v_minus]
    # 우리가 회로에서 쓸 유니터리: W^\dagger (Z측정 기준으로 보낼 것)
    return np.outer(V, V.conj())                   # W = V,  W|0>=v_plus

def naimark_U_from_E0(E0):
    # E1 = I - E0
    I = np.eye(2, dtype=complex)
    E1 = I - E0
    # 대각화로 sqrt 계산
    def sqrt_psd(M):
        vals, vecs = np.linalg.eigh(M)
        vals_clipped = np.clip(vals, 0.0, None)
        root = vecs @ np.diag(np.sqrt(vals_clipped)) @ vecs.conj().T
        return root

    R0 = sqrt_psd(E0)
    R1 = sqrt_psd(I - E0)

    # ancilla projectors (on A)
    P00 = np.array([[1,0],[0,0]], dtype=complex)  # |0><0|
    P11 = np.array([[0,0],[0,1]], dtype=complex)  # |1><1|
    P01 = np.array([[0,1],[0,0]], dtype=complex)  # |0><1|
    P10 = np.array([[0,0],[1,0]], dtype=complex)  # |1><0|

    # wires=[S, A] 기준에서 U(|ψ>⊗|0>) = (√E0|ψ>)⊗|0> + (√(I−E0)|ψ>)⊗|1>
    U = np.kron(R0, P00) + np.kron(R1, P10) - np.kron(R1, P01) + np.kron(R0, P11)
    return U

@qml.qnode(dev2)
def povm_via_naimark_probs(psi, E0):
    # 시스템 준비
    qml.StatePrep(psi, wires=S)
    # 보조 |0>
    # 유니터리 U 적용
    U = naimark_U_from_E0(E0)
    qml.QubitUnitary(U, wires=[S, A])
    # 보조 측정 확률 = [Pr(E0), Pr(E1)]
    return qml.probs(wires=A)

# 예시 사용:
psi0 = np.array([1,0], dtype=complex)                 # |0>
psi2 = np.array([0,1], dtype=complex)
psi1 = (np.array([1,1],complex)/np.sqrt(2))           # |+>
psi  = (np.array([np.sqrt(1.0), np.sqrt(0.0)],complex))  # 임의 입력
E0 = helstrom_E0(psi0, psi1)
print(povm_via_naimark_probs(psi0, E0))  # [Pr(E0), Pr(E1)]

Helstrom bound is  0.8535533905932737
[0.85355339 0.14644661]


## 0res_tests

(a) two mixed states
$$\rho_1=\cos^2\frac{\theta}{2}\,|0\rangle\!\langle0|+\sin^2\frac{\theta}{2}\,|1\rangle\!\langle1|=\tfrac12(I+\cos\theta\,Z), \\
\rho_2=\cos^2\frac{\theta}{2}\,|+\rangle\!\langle+|+\sin^2\frac{\theta}{2}\,|-\rangle\!\langle-|=\tfrac12\big(I+\cos\theta\,X\big)$$

In [3]:
import pennylane as qml
from pennylane import numpy as np
import torch
from shc_povm import rdm


dev = qml.device("lightning.qubit", wires=2)

def rho1(wires):
    theta = np.pi/5
    qml.RY(theta, wires=wires[0])
    qml.CNOT(wires=[wires[0],wires[1]])

def rho2(wires):
    theta = np.pi/6
    qml.RY(theta, wires=wires[0])
    qml.CNOT(wires=[wires[0],wires[1]])
    qml.Hadamard(wires[0])
    qml.Hadamard(wires[1])

@qml.qnode(dev)
def classical_rho1():
    rho1([0,1])
    return qml.density_matrix([0,1])

@qml.qnode(dev)
def classical_rho2():
    rho2([0,1])
    return qml.density_matrix([0,1])

init_states = torch.stack([
    torch.tensor(classical_rho1(), dtype=torch.complex64),
    torch.tensor(classical_rho2(), dtype=torch.complex64)
])

In [ ]:
def rho1(wires):
    theta = 0
    qml.RY(theta, wires=wires[0])
    # qml.CNOT(wires=[wires[0],wires[1]])

def rho2(wires):
    theta = np.pi/3
    qml.RY(theta, wires=wires[0])
    # qml.CNOT(wires=[wires[0],wires[1]])

In [3]:
def rho1(wires):
    qml.Hadamard(wires=wires[0])
    qml.PhaseShift(2*np.pi*0/3, wires=0)

def rho2(wires):
    qml.Hadamard(wires=wires[0])
    qml.PhaseShift(2*np.pi*1/3, wires=0)

def rho3(wires):
    qml.Hadamard(wires=wires[0])
    qml.PhaseShift(2*np.pi*2/3, wires=0)

In [5]:
from shc_povm import *

init_states = [rho1, rho2, rho3]
n_prep = 1
n_sys = 1
n_outcome = 3
wires = [0, 1, 2]

dev = qml.device('lightning.gpu', wires=len(wires))
# 일단 고정하는게 VQSD assumption -> TODO: Test
a_priori_probs = [1/n_outcome] * n_outcome

## Config
config = {
    "device": "cpu",
    # "device": "cuda" if torch.cuda.is_available() else "cpu",
    "dtype": torch.float32,
    "seed": 1,
}
## DataLoader

## Model
torch.manual_seed(config["seed"])
model = SingleQubitPOVM(n_prep, n_sys, n_outcome, config).to(device=config["device"], dtype=config["dtype"])
## criterion
criterion = tp_maximization_cost
## optimizer
opt = torch.optim.Adam(model.parameters(), lr=0.02)
## Trainer
povm_trainer = POVM_Torch_Trainer(n=n_outcome, a_priori_probs=a_priori_probs, optimizer=opt, model=model, criterion=criterion)
## Trainer.fit()
cost_list = povm_trainer.fit(X=init_states)
print(cost_list)
## Model
## load_weights()
## Model.__call__()


ValueError: RY: wrong number of wires. 2 wires given, 1 expected.

예시: **l-ary POVM**

$$\{\rho_i\}_{i=0}^{l-1}$$

In [21]:
import numpy as np
import pennylane as qml

def sqrt_psd(M, clip=0.0):
    # 수치 안전용 hermitize + 음수 고정
    Mh = 0.5*(M + M.conj().T)
    vals, vecs = np.linalg.eigh(Mh)
    vals = np.clip(vals.real, clip, None)
    return vecs @ np.diag(np.sqrt(vals)) @ vecs.conj().T

def complete_unitary_from_isometry(V, tol=1e-12):
    """V: (N x d),  V^† V = I_d 인 등거리 사상을 첫 d열로 유지한 채
    유니터리 U (N x N)로 완성."""
    N, d = V.shape
    cols = [V[:,j].copy() for j in range(d)]  # 첫 d열 그대로 유지
    def orth(v):
        for c in cols:
            v = v - c * (c.conj().T @ v)
        n = np.linalg.norm(v)
        return (v/n) if n > tol else None
    # 표준기저에서 보충
    for k in range(N):
        e = np.zeros(N, dtype=complex); e[k] = 1.0
        v = orth(e)
        if v is not None:
            cols.append(v)
        if len(cols) == N: break
    U = np.column_stack(cols)
    return U

def naimark_U_from_Es(E_list):
    """E_list: [E0, E1, ..., E_{m-1}] (각 dxd, PSD), ancilla는 qubit들로 임베드."""
    d = E_list[0].shape[0]
    m = len(E_list)
    # 큐비트 보조공간 크기
    k = int(np.ceil(np.log2(m)))
    M = 2**k
    Ks = [sqrt_psd(E) for E in E_list]
    # 남는 결과 채널(사용 안함) 0으로 패딩
    Ks += [np.zeros((d,d), dtype=complex) for _ in range(M - m)]
    # 등거리 사상 V 만들기
    V = np.vstack(Ks)  # (Md) x d
    # 수치 확인(선택)
    # assert np.allclose(V.conj().T @ V, np.eye(d), atol=1e-9)
    # 유니터리 완성
    U = complete_unitary_from_isometry(V)
    return U, k

# --- PennyLane 사용 예시 ---
# 시스템 n_sys_qubits, 보조 anc_k = k
def povm_probs_via_naimark(psi, E_list, sys_wires, anc_wires, dev):
    @qml.qnode(dev)
    def circuit():
        qml.StatePrep(psi, wires=sys_wires)
        U, k = naimark_U_from_Es(E_list)
        assert k == len(anc_wires), "anc_wires 길이가 k와 일치해야 합니다."
        qml.QubitUnitary(U, wires=(sys_wires + anc_wires))
        return qml.probs(wires=anc_wires)  # 길이 2^k 확률, 앞 m개가 E_i 확률
    return circuit()


In [22]:
n = 2
dev = qml.device("lightning.qubit", wires=n, shots=None)

# 예: 1-큐빗 trine POVM을 0번 와이어에 적용 (E_k = (1/3)I + (1/3) n_k · σ)
ops_tpl = [qml.Identity(0), qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]
nvecs = [
    (np.cos(0),           np.sin(0),           0.0),
    (np.cos(2*np.pi/3),   np.sin(2*np.pi/3),   0.0),
    (np.cos(4*np.pi/3),   np.sin(4*np.pi/3),   0.0),
]
Hks = [
    qml.Hamiltonian([1/3, (1/3)*nx, (1/3)*ny, (1/3)*nz], ops_tpl)
    for (nx, ny, nz) in nvecs
]

prjs = [0,1]

def prepare(theta):
    # 예시: 0번 큐빗에만 상태 준비
    qml.RY(theta, wires=0)

@qml.qnode(dev)
def povm_probs(theta):
    prepare(theta)
    qml.Hadamard(wires=1)
    return [qml.expval(qml.Projector([P], wires=[1])) for P in prjs]  # [p0, p1, p2]

for theta in np.linspace(0, 1, 11):
    p0, p1 = povm_probs(theta)
    print(p0, p1)
# 필요시 p2 = 1 - p0 - p1 로 줄일 수 있음

0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.49999999999999994 0.49999999999999994
0.49999999999999983 0.49999999999999983
0.49999999999999994 0.49999999999999994
0.4999999999999999 0.4999999999999999
0.49999999999999994 0.49999999999999994
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999
0.4999999999999999 0.4999999999999999


In [23]:
print(p0)

0.4999999999999999


In [24]:
# 디바이스 준비
dev = qml.device("default.qubit", wires=1, shots=1000)



# POVM 정의
E0 = 0.5 * qml.Projector([0], wires=0) + 0.5 * qml.Projector([1], wires=0)
E1 = qml.Identity(0) - E0
POVM = [E0, E1]

@qml.qnode(dev)
def povm_circuit(theta):
    qml.RX(theta, wires=0)
    return qml.sample(qml.POVM(POVM, wires=0))

print(povm_circuit(0.3))

AttributeError: module 'pennylane' has no attribute 'POVM'